In [ ]:
import json
import requests
from src import utils
import time
import os
from pprint import pprint
import random
from tqdm import tqdm

In [ ]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api-peft3.json'

In [ ]:
api = utils.read_json_file(api_path)

In [ ]:
# output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [ ]:
# os.makedirs(output_path, exist_ok=True)
# output_file_name = 'result-' + input_file.split('/')[-1]
# output_file = output_path + '/' + output_file_name
# print(f'output_file : {output_file}')

In [ ]:
# peft tunning
# peft1 = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft1-test.jsonl'
# peft2 = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft2-test.jsonl'

# ICL
# zs = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-public_mwp_data_v2_preprocess'
# zs_cot = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-cot-public_mwp_data_v2_preprocess'
# one_shot_cot = '/userhomes/philhoon/kt-ai-challenge/result/one-shot-cot-result-ko-gsm8k-test.jsonl'


In [ ]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [ ]:
model = model_api(api['url'], api['headers'])
BODY_TEMP = api['body']

In [ ]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [ ]:
def get_gsm8k_zs3(que):
    prompt = f'다음 질문에 차근차근 답하시오. {que}'
    return prompt

In [ ]:
## Test dataset

# GSM8K 8792 (7473/1319)
gsm8k = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'

# MWP_KR_DATA 3750
mwp_kr = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning-test.jsonl'

# AddSub 395
addsub = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_addsub-test.json'

# MultiArith 600
multiarith = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_multiarith-test.json'

# SingleEq 395
single_eq = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_single_eq-test.json'

In [ ]:
gsm8k_data = utils.read_jsonlines(gsm8k)
mwp_kr_data = utils.read_jsonlines(mwp_kr)
addsub_data = utils.read_jsonlines(addsub)
multiarith_data = utils.read_jsonlines(multiarith)
single_eq_data = utils.read_jsonlines(single_eq)

In [ ]:
print(len(gsm8k_data))
print(len(mwp_kr_data))
print(len(addsub_data))
print(len(multiarith_data))
print(len(single_eq_data))

In [ ]:
# random_indices

In [ ]:
data = single_eq_data

In [ ]:
data[0]

In [ ]:
for ins in tqdm(data):
    kor_que = ins['kor_question']
    prompt = get_gsm8k_zs3(kor_que)
    body = get_body(kor_que, BODY_TEMP)
    
#     kor_que = ins['question']
#     body = get_body(kor_que, BODY_TEMP)
    
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]

#     ins['zs_infer'] = res_context
    print(f"prompt : \n{prompt}")
    print(f"response : \n{res_context}")
    print(f"----------------------------")
#     print(f"reason : \n{ins['reason']}")
    print(f"answer : \n{ins['answer']}")
    print(f"=============================")
#     jsonl_writer.write_json_line(ins)

In [ ]:
zs_cot_data = utils.read_jsonlines(zs_cot)
zs_data = utils.read_jsonlines(zs)

In [ ]:
print(len(zs_cot_data))
len(zs_cot_data) == len(zs_data)

In [ ]:
from pprint import pprint

In [ ]:
cnt = 0
for ins in zs_data:
#     pprint(ins)
    print(f"question :\n{ins['question']}")
#     print(f"zs_cot_infer :\n{ins['zs_cot_infer']}")
    print(f"zs_infer :\n{ins['zs_infer']}")
    print(f"answer :\n{ins['answer']}")
    print("===========================")
    cnt += 1
    if cnt == 50:
        break